<a href="https://colab.research.google.com/github/dhrits/LLM-Engineering-Foundations-to-SLMs/blob/main/04_Embeddings/Training_Emebddings_From_Scratch_%26_Fine_tuning_Embedding_Models_Coding_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Embeddings & You - A Brief Introduction to Embeddings in Machine Learning

If you've toyed with LangChain, LlamaIndex, or even OpenAI's `ada` model - you've likely run into the word: "Embeddings" a few time.

They've had a recent surge in popularity due to the profliferation of Retrieval Augmented Generation, but they've been around for a very long time.

If you come from an NLP background, embeddings are something you might be intimately familiar with - otherwise, you might find the topic a bit...dense. (this attempt at a joke will make more sense later)

In all seriousness, embeddings are a powerful piece of the NLP puzzle, so let's dive in!

> NOTE: While this notebook language/NLP-centric, embeddings have uses beyond just text!

## Notebook Table of Contents:

- Breakout Room #1: Training Word2Vec from Scratch
  - Task 1: Dependencies
  - Task 2: Data Collection
  - Task 3: Data Preprocessing
    - 🏗️ Activity #1
    - ❓Question #1
    - 🏗️ Activity #2
    - 👪❓ Discussion Question #1
  - Task 4: Training Word2Vec
    - 🏗️ Activity #3
    - ❓Question #2
- Breakout Room #2:
  - Task 1: Fine-tuning Our Embedding Model
    - ❓Question #3
    - 🏗️ Activity #4
  - Task 2: Evaluating our Embedding Model
    - 👪❓Discussion Question #2

### Why Do We Even Need Embeddings?

In order to fully understand what Embeddings are, we first need to understand why we have them:

Machine Learning algorithms, ranging from the very big to the very small, all have one thing in common:

*They need numeric inputs.*

So we need a process by which to translate the domain we live in, dominated by images, audio, language, and more, into the domain of the machine: Numbers.

Another thing we want to be able to do is capture "semantic information" about words/phrases so that we can use algorithmic approaches to determine if words are closely related or not!

So, we need to come up with a process that does these two things well:

1. Convert non-numeric data into numeric-data
2. Capture potential semantic relationships between individual pieces of data

## Breakout Room #1: Training Word2Vec from Scratch

Now that we have a bit of background on Embeddings - let's look at what it takes to create our own embeddings using Word2Vec!

We'll be leveraging the `gensim` library, which you can read all about [here](https://pypi.org/project/gensim/).

Before we begin training, however, we need some data!

Let's use the Wikipedia pages for Wicked and Gladiator as examples.

### Task 1: Dependencies
We'll leverage the `wikipedia` library, and `langchain`s `WikipediaLoader` to obtain our Wikipedia data!

In [ ]:
!pip install -U -q wikipedia langchain langchain_community lxml datasets

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 27.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 81.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 34.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.5/409.5 kB 35.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 4.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 20

> NOTE: Please reset the Colab environment after running the install cells.

### Task 2: Data Collection



In [ ]:
from langchain_community.document_loaders import WikipediaLoader

wicked_docs = WikipediaLoader(
    query="Wicked (2024 film)",
    load_max_docs=5,
    doc_content_chars_max=1_000_000
    ).load()

/usr/local/lib/python3.10/dist-packages/wikipedia/wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /usr/local/lib/python3.10/dist-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


In [ ]:
len(wicked_docs)

4

In [ ]:
gladiator_2_docs = WikipediaLoader(
    query="Gladiator II",
    load_max_docs=5,
    doc_content_chars_max=1_000_000
    ).load()

In [ ]:
len(gladiator_2_docs)

5

### Task 3: Data Preprocessing

Now that we have some text, we need to do some preprocessing! That's right - classic NLP!

Let's begin by cleaning up our text, we'll:

- Remove special characters
- Remove stop words
- Remove links
- Convert to lowercase
- Strip whitespace

To do this, we'll need two main modules:

- The `re` standard library module
- `spacy`, another NLP library

In [ ]:
import nltk

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('punkt_tab')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

Let's take a peek at what these "stopwords" are - for traditional embedding models and NLP.

In [ ]:
from nltk.corpus import stopwords
stopwords.words('english')[:10]

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're"]

#### Text Normalization

The first step is to make a helper function that normalizes our text.

#####🏗️ Activity #1:

What should the output format of the `preprocess_text` function be?

**`preprocess_text` should strip out some special characters and essentially output a list of lowercased words (our tokens).**

Once you've determined the output format - please complete the code cell and ensure the appropriate format is returned.

In [ ]:
import re
from typing import List
from nltk.tokenize import word_tokenize

def preprocess_text(text: str) -> List[str]:
  ### YOUR CODE HERE
  # Remove urls https://stackoverflow.com/questions/6718633/python-regular-expression-again-match-url
  text = re.sub(r"""((http|https)\:\/\/)?[a-zA-Z0-9\.\/\?\:@\-_=#]+\.([a-zA-Z]){2,6}([a-zA-Z0-9\.\&\/\?\:@\-_=#])*""", "", text)
  # Remove non alphanumeric characters with space
  text = re.sub(r"[^a-zA-Z]", " ", text)
  # lower case
  text = text.lower().strip()
  # tokenize based on words
  text = word_tokenize(text)
  filtered_tokens = [token for token in text if token not in stopwords.words('english')]
  return filtered_tokens

Let's see how this works on some of our Wikipedia data!

In [ ]:
preprocess_text(wicked_docs[0].page_content[:100])

['wicked',
 'titled',
 'onscreen',
 'wicked',
 'part',
 'american',
 'epic',
 'musical',
 'fantasy',
 'film',
 'directed']

#### Sentence Tokenization:

Now we'll turn our corpus into sets of sentences.

#####🏗️ Activity #2:

What should the output format of the `sentence_tokenization` function be?

**`sentence_tokenization` function should take several sentences (`text`) and output a list of lists where each individual list represents the sentence tokenized into individual tokens (lower cased words) and the outer list represents the list of all sentences.**

Once you've determined the output format - please complete the code cell and ensure the appropriate format is returned.

> NOTE: We've mysteriously imported the `sent_tokenize` helper function - it may be useful. Check out the [docs](https://www.nltk.org/api/nltk.tokenize.sent_tokenize.html)!

In [ ]:
from nltk.tokenize import sent_tokenize

def sentence_tokenization(text: str) -> List[List[str]]:
    sentences = sent_tokenize(text)
    sentence_tokens = [preprocess_text(sentence) for sentence in sentences]
    return sentence_tokens

In [ ]:
sentence_tokenization(wicked_docs[0].page_content[:200])

[['wicked',
  'titled',
  'onscreen',
  'wicked',
  'part',
  'american',
  'epic',
  'musical',
  'fantasy',
  'film',
  'directed',
  'jon',
  'chu',
  'written',
  'winnie',
  'holzman',
  'dana',
  'fox',
  'songs',
  'stephen',
  'schwartz'],
 ['fir']]

Perfect, with that, we're ready to create our corpus!

In [ ]:
corpus = []

for doc in wicked_docs:
  corpus += sentence_tokenization(doc.page_content)

for doc in gladiator_2_docs:
  corpus += sentence_tokenization(doc.page_content)

##### ❓ Question #1:

Why is this normalization and tokenization necessary to train a Word2Vec Embedding Model?

Let's tackle each stage:

* **Normalization** - Normalization has several benefits in this case.

    1. Avoids an overly large vocabulary - Without good normalization, the vocabulary for training an embedding model might get prohibitively large. This can have implications on the amount of compute needed to train the model.
    2. Avoids overfitting by reducing "noise" - Often, normalization can get rid of variations of words which have very similar meanings (this can be via lowercasing, stemming, lemmatization etc) which can avoid overfitting since the model doesn't have to do extra work to understand subtle differences in usages.

* **Tokenization**
    1. Word2Vec is essentially a "word" embedding model which must operate on some discrete units (words). Tokenization is the process of obtaining these discrete units. Once they've been determined, we can learn the meaning of these discrete units using a neural network and from the context in which they appear.
    2. Tokenization is important because most language models operate at the level of discrete objects like tokens (as an example, for a next-word predictor, the softmax can be over tokens). Not having any tokens would require either building a character level model (which is still arguably tokenizing but at the character level) or come up with some novel method which operates on sentences. Note that tokens don't have to be words (see next activity).

##### 👪❓ Discussion Question #1:

When creating training data for Large Language Models, do we need to/should we use text normalization?

What arguments for or against text normalization exist at LLM-scale datasets?

The short answer is that modern language models have enough capacity and are trained using sufficient compute that normalization isn't *strictly necessary* but can still be useful depending on the application. To explain this further, Byte Pair Encoding tokenizer (originally used in GPT) can be configured with a target vocabulary size and use statistical techniques to determine the right tokens given the training corpus. These tokens are usually sub-word (pieces of words) determined at the byte-level. This technique can deal with nuances of a larger vocabulary without tokenization.

Arguments for Normalization:
1. Depending on the size of the "large" language model, normalization can still help reduce the vocabulary size (allowing accelerator flops to be used for other purposes).
2. Depending on the context, once again, normalization might help the ease the training process by avoiding overfitting on unnecessary variations of various tokens.

Arguments against Normalization:
1. Some kinds of normalizations (like removal of extra spaces) can be detrimental depending on the application. As an example, a language model built to write Python code needs to be able to understand the meaning behind number of spaces.
2. Depending on the kinds of normalization applied, the training pipeline for the model can become overly complicated. Large Language Models typicall have enough capacity and compute that this complexity is not worth the added trouble.
3. A general purpose LLM should be able to understand nuances of language which normalization can destroy. As an example, a language model might be asked to correct grammar or write essays which require grasp of language constructs which normalization can impact.

### Task 4: Training Word2Vec

Now that we have our corpus set up, we can train our Word2Vec model.

Training is straightforward, thanks to `gensim`, and more can be understood about the process by reading the paper - but let's see it in code!

It's also worth considering/playing around with the `gensim` parameters.

In [ ]:
!pip install -q -U gensim

### An Aside on Skip-gram (SG) and Continuous Bag of Words (CBOW):

**Skip-gram**:

Skip-gram is an approach to teaching computers the meaning of words by predicting the surrounding context from a given word. Think of it as a student who learns by taking a single word and trying to guess what words might appear around it. For example, given the word "sun," Skip-gram would learn to predict related words like "bright," "sky," and "shine." This method is particularly effective at handling rare words in the vocabulary and capturing multiple meanings of words, though it typically requires more training time. The key insight is that words appearing in similar contexts often have related meanings.

**Continuous Bag of Words (CBOW)**:

CBOW takes the opposite approach to Skip-gram by predicting a target word based on its surrounding context words. Imagine playing a fill-in-the-blank game where you see "The ___ is barking at the mailman" and need to predict "dog" based on the surrounding words. CBOW looks at multiple context words at once and tries to understand what word would make sense in the middle. This method tends to be faster to train than Skip-gram and performs particularly well with frequent words in the vocabulary. However, it might not be as effective at handling rare words or capturing multiple word meanings since it averages the context.

#####🏗️ Activity #3:

Set appropriate hyperparameters for the gensim `Word2Vec` model.

> NOTE: Documentation is available [here](https://radimrehurek.com/gensim/models/word2vec.html#gensim.models.word2vec.Word2Vec)


`VECTOR_SIZE` - This is the size of the embedding produced by the model. I used 256 because it was still reasonable compute time and empirically seemed to produce reasonable results.

`WINDOW` - This is the maximum distance between current and predicted word. The size of 10 seemed appropriate here as it is roughly average sentence length.

`MIN_COUNT` - Words with frequency lower than this in the corpus are ignored. Since this corpus is very specific to two genre movies, I kept this to 2 since rare words (which are highly context specific) are expected in the limited corpus.

##### ❓ Question #2:

What do each of the hyper-parameters mean or relate to:

- `VECTOR_SIZE` -> This is the size of the embedding produced by the model.
- `WINDOW` -> This is the maximum distance between current and predicted word used to set up the prediction problem.
- `MIN_COUNT` -> Words with frequency lower than this in the corpus are ignored.
- `SG` -> Specifies the algorithm used - 1 for Skipgram, otherwise CBOW. We specify 1 because we want to use skipgram.

In [ ]:
from gensim.models import Word2Vec

VECTOR_SIZE = 256
WINDOW = 10
MIN_COUNT = 2
SG = 1

model = Word2Vec(
    sentences=corpus,
    vector_size=VECTOR_SIZE,
    window=WINDOW,
    min_count=MIN_COUNT,
    sg=SG
    )

Blink and you'll miss it. You just trained an embeddings model!

Let's try it out and see what we did!

In [ ]:
model.wv["elphaba"]

array([ 0.06765869,  0.00931335,  0.05225882,  0.2788173 ,  0.15067585,
        0.00614858,  0.10582946,  0.08191479, -0.00182309,  0.06236859,
        0.05381269, -0.03466383,  0.1107581 ,  0.0188936 , -0.0485948 ,
        0.07580302,  0.03940454, -0.02565183, -0.1804806 ,  0.29081035,
        0.09020197,  0.10945549, -0.00210144, -0.22987579, -0.09499752,
        0.10487061,  0.00490394, -0.03315461, -0.10035457, -0.05638117,
        0.16126363,  0.0371501 , -0.10562462,  0.03407912,  0.07660402,
        0.1364852 ,  0.15054387, -0.01378694,  0.05009715, -0.09255938,
       -0.07831604,  0.04636762, -0.01723669, -0.00755542,  0.2774938 ,
       -0.0349294 ,  0.10037494, -0.024217  ,  0.00737526,  0.04616126,
       -0.09552637,  0.0279586 , -0.0165402 ,  0.02621889, -0.10678325,
       -0.08180283,  0.03920405, -0.24745321, -0.00959589,  0.13088323,
       -0.10962094, -0.02462117, -0.00329851,  0.11779118,  0.07615861,
       -0.00654701, -0.06600293,  0.19846079, -0.03862818, -0.01

Finally! We see it: An embedding in the wild.

Notice how we input a word, in this case "Elphaba", and we got back a 100-dimensional vector of floats.

Let's see if we can't get back a list of similar vectors to the vector for "Elphaba", and "Maximus"!

In [ ]:
model.wv.most_similar(positive=["elphaba"], topn=3)

[('glinda', 0.9986661076545715),
 ('galinda', 0.9972348809242249),
 ('wizard', 0.9971489906311035)]

In [ ]:
model.wv.most_similar(positive=["maximus"], topn=3)

[('lucilla', 0.998903751373291),
 ('son', 0.998900830745697),
 ('aurelius', 0.9986047148704529)]

Now, for the moment of truth - let's do some vector math and see what happens!

In [ ]:
galinda_vec = model.wv["galinda"]
good_vec = model.wv["good"]
mystery_vector = galinda_vec - good_vec

In [ ]:
model.wv.most_similar(positive=[mystery_vector], topn=3)

[('elphaba', 0.7351096868515015),
 ('glinda', 0.7306303381919861),
 ('galinda', 0.7229472994804382)]

And there we have it - embeddings, and a demonstration of what makes them so powerful!

> Note: This is a very small sample size, and while this result is what we'd hope for - it is largely coincidental - this behaviour is expressed better in much larger corpus' of text.

## Breakout Room #2: Fine-tuning a BERT-Style Embedding Model on Question Answer Pairs.

Now that we've seen where embeddings "started", as it were, let's see where they've gotten.

In this section, we'll be fine-tuning Hugging Face's [sentence transformers](https://www.sbert.net/).

Sentence Transformers leverages the work done in the [Sentence-BERT](https://arxiv.org/abs/1908.10084) paper. So while the idea of converting input text into a dense vector representation is the same, the way we got to those embeddings is a bit different.

> NOTE: As the name implies, the following model is an *ENTIRE* transformer model (though Encoder-only, as described by Sentence-BERT).

### Fine-tuning Our Embeddings Model

Finally, the set up is complete - and we can move on to fine-tuning our sentence transformer embedding model!

The process is simplified considerably by how amazing the Hugging Face `sentence-transformer` library is, so let's jump straight in!

In [ ]:
!pip install -U -q sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 6.4 MB/s eta 0:00:00


In [ ]:
from sentence_transformers import SentenceTransformer

We're going to use the `BAAI/bge-small-en` embedding model as an example, but you could use any of the `sentence-transformer` embeddings models.

In [ ]:
model_id = "BAAI/bge-small-en"
model = SentenceTransformer(model_id)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/90.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
model

SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': True}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': True, 'pooling_mode_mean_tokens': False, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
)

Let's load our data into the desired format!

In [ ]:
from torch.utils.data import DataLoader
from sentence_transformers import InputExample

In [ ]:
!git clone https://github.com/AI-Maker-Space/DataRepository

Cloning into 'DataRepository'...
remote: Enumerating objects: 119, done.
remote: Counting objects: 100% (111/111), done.
remote: Compressing objects: 100% (96/96), done.
remote: Total 119 (delta 36), reused 40 (delta 10), pack-reused 8 (from 1)
Receiving objects: 100% (119/119), 78.04 MiB | 26.82 MiB/s, done.
Resolving deltas: 100% (36/36), done.


In [ ]:
TRAIN_DATASET_FPATH = './DataRepository/embedding_data/train_dataset.json'
VAL_DATASET_FPATH = './DataRepository/embedding_data/eval_dataset.json'

In [ ]:
import json

with open(TRAIN_DATASET_FPATH, 'r+') as f:
    train_dataset = json.load(f)

with open(VAL_DATASET_FPATH, 'r+') as f:
    val_dataset = json.load(f)

In [ ]:
dataset = train_dataset

corpus = dataset['corpus']
queries = dataset['queries']
relevant_docs = dataset['relevant_docs']

examples = []
for query_id, query in queries.items():
    node_id = relevant_docs[query_id][0]
    text = corpus[node_id]
    example = InputExample(texts=[query, text])
    examples.append(example)

We're going to be leveraging `sentence_transformers` `MultipleNegativesRankingLoss` as our loss function.

You can read more about it in the docs, [here](https://www.sbert.net/docs/package_reference/losses.html#multiplenegativesrankingloss).

Note that there is [research](https://arxiv.org/pdf/1705.00652.pdf) that indicates that performance generally scales with `BATCH_SIZE`, but we're going to stick with an arbitrary 10 for the example in the notebook.

##### ❓ Question #3:

What is happening in `MultipleNegativesRankingLoss` that makes it useful for our task?

`MultipleNegativesRankingLoss` is a form of *contastive loss* which aims to bring tokens which are similar "closer together" while also "increasing the distance" between tokens which are less similar by making use of multiple dissimilar (negative tokens) to create this contrast.

This implementation is especially useful if we only have positive pairs (which is the case in this training dataset). The implementation will then use the other tuples in the batch as an examples of a negative with the current tuple. As a simplified example, for a batch size of 10, if each pair is a positive example, then the other 9 tuples may be used to extract negative examples.

In [ ]:
from sentence_transformers import losses

In [ ]:
loss = losses.MultipleNegativesRankingLoss(model)

In [ ]:
BATCH_SIZE = 10

loader = DataLoader(
    examples, batch_size=BATCH_SIZE
)

We'll set up the `InformationRetrievalEvaluator` to determine performance during training.

In [ ]:
from sentence_transformers.evaluation import InformationRetrievalEvaluator

In [ ]:
dataset = val_dataset

corpus = dataset['corpus']
queries = dataset['queries']
relevant_docs = dataset['relevant_docs']

evaluator = InformationRetrievalEvaluator(queries, corpus, relevant_docs)

You could use a larger epoch size here, but for the example in the Notebook, we'll stick with 10.

In [ ]:
EPOCHS = 10

Nothing left to do but #trainthatmodel!

> NOTE: You'll need to make sure you enter the desired Weights and Biases key - you should be able to simple click the link `https://wandb.ai/authorize` and follow the outlined steps to get the API key.

In [ ]:
# wandb terms of service and marketing folks are quite intrusive and require Company of Work (and thereafter reach out to work). While I've set up
# this account, I've disabled it for this assignment for this reason.
import os
os.environ["WANDB_DISABLED"] = "true"

In [ ]:
from datasets import Dataset
from torch.utils.data import DataLoader

warmup_steps = int(len(loader) * EPOCHS * 0.1)

model.fit(
    train_objectives=[(loader, loss)],
    epochs=EPOCHS,
    warmup_steps=warmup_steps,
    output_path='exp_finetune',
    show_progress_bar=True,
    evaluator=evaluator,
    evaluation_steps=50,
)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Step,Training Loss,Validation Loss,Cosine Accuracy@1,Cosine Accuracy@3,Cosine Accuracy@5,Cosine Accuracy@10,Cosine Precision@1,Cosine Precision@3,Cosine Precision@5,Cosine Precision@10,Cosine Recall@1,Cosine Recall@3,Cosine Recall@5,Cosine Recall@10,Cosine Ndcg@10,Cosine Mrr@10,Cosine Map@100
50,No log,No log,0.630872,0.791946,0.838926,0.899329,0.630872,0.263982,0.167785,0.089933,0.630872,0.791946,0.838926,0.899329,0.763430,0.720113,0.724508
60,No log,No log,0.627517,0.778523,0.842282,0.885906,0.627517,0.259508,0.168456,0.088591,0.627517,0.778523,0.842282,0.885906,0.755411,0.713607,0.718658
100,No log,No log,0.644295,0.788591,0.845638,0.895973,0.644295,0.262864,0.169128,0.089597,0.644295,0.788591,0.845638,0.895973,0.767730,0.726899,0.731367
120,No log,No log,0.634228,0.788591,0.832215,0.895973,0.634228,0.262864,0.166443,0.089597,0.634228,0.788591,0.832215,0.895973,0.764712,0.722739,0.727859
150,No log,No log,0.630872,0.771812,0.835570,0.885906,0.630872,0.257271,0.167114,0.088591,0.630872,0.771812,0.835570,0.885906,0.756317,0.714995,0.720317
180,No log,No log,0.637584,0.798658,0.832215,0.902685,0.637584,0.266219,0.166443,0.090268,0.637584,0.798658,0.832215,0.902685,0.767390,0.724489,0.728832
200,No log,No log,0.630872,0.795302,0.835570,0.906040,0.630872,0.265101,0.167114,0.090604,0.630872,0.795302,0.835570,0.906040,0.766550,0.722321,0.726448
240,No log,No log,0.627517,0.778523,0.812081,0.889262,0.627517,0.259508,0.162416,0.088926,0.627517,0.778523,0.812081,0.889262,0.755208,0.712708,0.717628
250,No log,No log,0.634228,0.778523,0.815436,0.889262,0.634228,0.259508,0.163087,0.088926,0.634228,0.778523,0.815436,0.889262,0.758341,0.716775,0.722066
300,No log,No log,0.637584,0.791946,0.838926,0.895973,0.637584,0.263982,0.167785,0.089597,0.637584,0.791946,0.838926,0.895973,0.764657,0.722772,0.727183


Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

### Task 2: Evaluating our Embeddings Models

Now that we've fine-tuned our embedding model on our data - lets see how it performs compared to the base embeddings!

In [ ]:
import json
from tqdm.notebook import tqdm
import pandas as pd

In [ ]:
TRAIN_DATASET_FPATH = './DataRepository/embedding_data/train_dataset.json'
EVAL_DATASET_FPATH = './DataRepository/embedding_data/eval_dataset.json'

In [ ]:
with open(TRAIN_DATASET_FPATH, 'r+') as f:
    train_dataset = json.load(f)

with open(EVAL_DATASET_FPATH, 'r+') as f:
    eval_dataset = json.load(f)

We're going to be using the `InformationRetrievalEvaluator` to help us determine how well our embedding model is performing on a widely used task: Information Retrieval!

You can dive deeper into the documentation [here](https://sbert.net/docs/package_reference/sentence_transformer/evaluation.html#informationretrievalevaluator) to see under the hood.

You'll notice, however, that we have common suffixes for our evaluation metrics:

- `X_accuracy@1`, `X_accuracy@3`, etc.

This is computing metrics by looking at the accuracy, recall, precision, Mean Reciprocal Rank (MRR), Normalized Discounted Cumulative Gain (NDGC), and Mean-Average Precision (MAP) at various numbers of retrieved items.

That is to say:

We look at these scores as we include the first closest document, top three closest documents, etc.

We can think of these `@k` as "top k` metrics.

These will help us guide important hyper-parameters when using these models for Information Retrieval tasks down the road!

In [ ]:
from sentence_transformers.evaluation import InformationRetrievalEvaluator
from sentence_transformers import SentenceTransformer

def evaluate_st(
    dataset,
    model_id,
    name,
):
    corpus = dataset['corpus']
    queries = dataset['queries']
    relevant_docs = dataset['relevant_docs']

    evaluator = InformationRetrievalEvaluator(queries, corpus, relevant_docs, name=name)
    model = SentenceTransformer(model_id)
    return evaluator(model, output_path="/content/")

#####🏗️ Activity #4:

Describe what the `evaluate` function is doing in the above cell in natural language.

Above, the "evaluate" function is evaluating various metrics like accuracy, precision and recall (see definitions below) by fetching most similar items as determined by cosine similarity of the embeddings. However, the @k distinction specifies whether the measure of a "match" is only with the top fetched output (eg @1) or ANY of the top k matches.


Precision is the fraction of retrieved docs which are relevant amongst all retrieved docs.

Recall is the fraction of retrieved docs which are relevant amongst all relevant docs (including those not fetched by the model).

#### Base Embeddings Model Results

In [ ]:
evaluate_st(eval_dataset, "BAAI/bge-small-en", name='bge')

{'bge_cosine_accuracy@1': 0.5067114093959731,
 'bge_cosine_accuracy@3': 0.714765100671141,
 'bge_cosine_accuracy@5': 0.7818791946308725,
 'bge_cosine_accuracy@10': 0.8288590604026845,
 'bge_cosine_precision@1': 0.5067114093959731,
 'bge_cosine_precision@3': 0.23825503355704697,
 'bge_cosine_precision@5': 0.1563758389261745,
 'bge_cosine_precision@10': 0.08288590604026844,
 'bge_cosine_recall@1': 0.5067114093959731,
 'bge_cosine_recall@3': 0.714765100671141,
 'bge_cosine_recall@5': 0.7818791946308725,
 'bge_cosine_recall@10': 0.8288590604026845,
 'bge_cosine_ndcg@10': 0.6710313851865369,
 'bge_cosine_mrr@10': 0.619814637264302,
 'bge_cosine_map@100': 0.6279603491960256}

#### Fine-tuned Results

In [ ]:
evaluate_st(eval_dataset, "exp_finetune", name='finetuned')

{'finetuned_cosine_accuracy@1': 0.6442953020134228,
 'finetuned_cosine_accuracy@3': 0.7885906040268457,
 'finetuned_cosine_accuracy@5': 0.8456375838926175,
 'finetuned_cosine_accuracy@10': 0.8959731543624161,
 'finetuned_cosine_precision@1': 0.6442953020134228,
 'finetuned_cosine_precision@3': 0.2628635346756152,
 'finetuned_cosine_precision@5': 0.16912751677852347,
 'finetuned_cosine_precision@10': 0.0895973154362416,
 'finetuned_cosine_recall@1': 0.6442953020134228,
 'finetuned_cosine_recall@3': 0.7885906040268457,
 'finetuned_cosine_recall@5': 0.8456375838926175,
 'finetuned_cosine_recall@10': 0.8959731543624161,
 'finetuned_cosine_ndcg@10': 0.7677298263110102,
 'finetuned_cosine_mrr@10': 0.7268989027378289,
 'finetuned_cosine_map@100': 0.7313671834327362}

### Conclusion

Now we can compare the embeddings models to see which performed the best!

In [ ]:
df_st_bge = pd.read_csv('/content/Information-Retrieval_evaluation_bge_results.csv')
df_st_finetuned = pd.read_csv('/content/Information-Retrieval_evaluation_finetuned_results.csv')

In [ ]:
df_st_bge['model'] = 'bge'
df_st_finetuned['model'] = 'fine_tuned'
df_st_all = pd.concat([df_st_bge, df_st_finetuned])
df_st_all = df_st_all.set_index('model')
df_st_all

,epoch,steps,cosine-Accuracy@1,cosine-Accuracy@3,cosine-Accuracy@5,cosine-Accuracy@10,cosine-Precision@1,cosine-Recall@1,cosine-Precision@3,cosine-Recall@3,cosine-Precision@5,cosine-Recall@5,cosine-Precision@10,cosine-Recall@10,cosine-MRR@10,cosine-NDCG@10,cosine-MAP@100
model,,,,,,,,,,,,,,,,,
bge,-1,-1,0.506711,0.714765,0.781879,0.828859,0.506711,0.506711,0.238255,0.714765,0.156376,0.781879,0.082886,0.828859,0.619815,0.671031,0.627960
fine_tuned,-1,-1,0.644295,0.788591,0.845638,0.895973,0.644295,0.644295,0.262864,0.788591,0.169128,0.845638,0.089597,0.895973,0.726899,0.767730,0.731367


##### 👪❓Discussion Question #2:

Discuss the results with your group!

Looking at the table above, for almost every single metric, the finetuned model performs better than the base model.